# Chicago Data Analysis with SQLite and SQL Magic

## Overview
This project analyzes real-world datasets related to the City of Chicago using SQLite and SQL magic in Jupyter Notebook.  
We use Python, Pandas, and SQL queries to store, explore, and answer specific analytical questions about crime, census data, and public schools.

## Datasets Used
1. **CENSUS_DATA**  
   - Contains census information for Chicago’s community areas, including population, per capita income, hardship index, and percentage of households below the poverty line.  

2. **CHICAGO_PUBLIC_SCHOOLS**  
   - Contains performance and other related data for public schools in Chicago, including safety scores and school type.

3. **CHICAGO_CRIME_DATA**  
   - Contains reported crime data in Chicago, including case numbers, crime types, location descriptions, and community area numbers.

## Objectives
- Load datasets into an SQLite database (`FinalDB.db`).
- Perform SQL queries to answer various analytical questions.
- Use subqueries, aggregation, filtering, and grouping to extract insights.

## Tools and Technologies
- **Python** (Pandas, sqlite3)
- **SQLite**
- **ipython-sql** (`%sql` magic)
- **Jupyter Notebook**

---


In [1]:
#!pip install pandas
#!pip install ipython-sql prettytable 
#pip install --upgrade prettytable


In [2]:
import prettytable

prettytable.DEFAULT = 'DEFAULT'

In [3]:
# 1️ Load pandas and sqlite3 libraries and establish a connection to FinalDB.db
import pandas as pd
import sqlite3

# Create/connect to database file
conn = sqlite3.connect('FinalDB.db')


In [4]:
# 2️ Load the SQL magic module
%load_ext sql
# fix style issue to avoid KeyError


In [5]:
# 3️ Establish a connection between SQL magic module and the database FinalDB.db
%sql sqlite:///FinalDB.db


In [6]:
# 4️ Use Pandas to load the data available in the links above to dataframes

census_url = "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DB0201EN-SkillsNetwork/labs/FinalModule_Coursera_V5/data/ChicagoCensusData.csv?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDeveloperSkillsNetworkDB0201ENSkillsNetwork20127838-2021-01-01"
schools_url = "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DB0201EN-SkillsNetwork/labs/FinalModule_Coursera_V5/data/ChicagoPublicSchools.csv?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDeveloperSkillsNetworkDB0201ENSkillsNetwork20127838-2021-01-01"
crime_url = "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DB0201EN-SkillsNetwork/labs/FinalModule_Coursera_V5/data/ChicagoCrimeData.csv?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDeveloperSkillsNetworkDB0201ENSkillsNetwork20127838-2021-01-01"

df_census = pd.read_csv(census_url)
df_schools = pd.read_csv(schools_url)
df_crime = pd.read_csv(crime_url)


In [7]:
# 5️ Use these dataframes to load data onto the database FinalDB.db as required tables

df_census.to_sql('CENSUS_DATA', conn, if_exists='replace', index=False)
df_schools.to_sql('CHICAGO_PUBLIC_SCHOOLS', conn, if_exists='replace', index=False)
df_crime.to_sql('CHICAGO_CRIME_DATA', conn, if_exists='replace', index=False)


533

In [8]:
result = %sql SELECT name FROM sqlite_master WHERE type='table';
df = result.DataFrame()


 * sqlite:///FinalDB.db
Done.


In [9]:
print(df)

                     name
0             CENSUS_DATA
1  CHICAGO_PUBLIC_SCHOOLS
2      CHICAGO_CRIME_DATA


In [10]:
df.head()

,name
0,CENSUS_DATA
1,CHICAGO_PUBLIC_SCHOOLS
2,CHICAGO_CRIME_DATA


### Problem 1
##### Find the total number of crimes recorded in the CRIME table

In [11]:
%sql SELECT COUNT(*) AS total_crimes FROM CHICAGO_CRIME_DATA;

 * sqlite:///FinalDB.db
Done.


total_crimes
533


### Problem 2
##### List community area names and numbers with per capita income less than 11000

In [12]:
%sql SELECT COMMUNITY_AREA_NAME, COMMUNITY_AREA_NUMBER \
     FROM CENSUS_DATA \
     WHERE PER_CAPITA_INCOME < 11000;


 * sqlite:///FinalDB.db
Done.


COMMUNITY_AREA_NAME,COMMUNITY_AREA_NUMBER
West Garfield Park,26.0
South Lawndale,30.0
Fuller Park,37.0
Riverdale,54.0


### Problem 3
##### List all case numbers for crimes involving minors?
##### (children are not considered minors for the purposes of crime analysis)

In [13]:
%sql SELECT CASE_NUMBER \
     FROM CHICAGO_CRIME_DATA \
     WHERE DESCRIPTION LIKE '%MINOR%' \
       AND DESCRIPTION NOT LIKE '%CHILD%';


 * sqlite:///FinalDB.db
Done.


CASE_NUMBER
HL266884
HK238408


### Problem 4
##### List all kidnapping crimes involving a child?

In [14]:
%sql SELECT *   \
     FROM CHICAGO_CRIME_DATA \
     WHERE PRIMARY_TYPE = 'KIDNAPPING' \
       AND DESCRIPTION LIKE '%CHILD%';


 * sqlite:///FinalDB.db
Done.


ID,CASE_NUMBER,DATE,BLOCK,IUCR,PRIMARY_TYPE,DESCRIPTION,LOCATION_DESCRIPTION,ARREST,DOMESTIC,BEAT,DISTRICT,WARD,COMMUNITY_AREA_NUMBER,FBICODE,X_COORDINATE,Y_COORDINATE,YEAR,LATITUDE,LONGITUDE,LOCATION
5276766,HN144152,2007-01-26,050XX W VAN BUREN ST,1792,KIDNAPPING,CHILD ABDUCTION/STRANGER,STREET,0,0,1533,15,29.0,25.0,20,1143050.0,1897546.0,2007,41.87490841,-87.75024931,"(41.874908413, -87.750249307)"


### Problem 5
##### List the kind of crimes that were recorded at schools. (No repetitions)

In [15]:
%sql SELECT DISTINCT PRIMARY_TYPE \
     FROM CHICAGO_CRIME_DATA \
     WHERE LOCATION_DESCRIPTION LIKE '%SCHOOL%';


 * sqlite:///FinalDB.db
Done.


PRIMARY_TYPE
BATTERY
CRIMINAL DAMAGE
NARCOTICS
ASSAULT
CRIMINAL TRESPASS
PUBLIC PEACE VIOLATION


### Problem 6
##### List the type of schools along with the average safety score for each type

In [16]:
%sql SELECT "Elementary, Middle, or High School" AS school_type, \
           AVG(SAFETY_SCORE) AS avg_safety_score \
     FROM CHICAGO_PUBLIC_SCHOOLS \
     GROUP BY "Elementary, Middle, or High School";


 * sqlite:///FinalDB.db
Done.


school_type,avg_safety_score
ES,49.52038369304557
HS,49.62352941176471
MS,48.0


### Problem 7
##### List 5 community areas with highest % of households below poverty line

In [17]:
%sql SELECT COMMUNITY_AREA_NAME,   \
           PERCENT_HOUSEHOLDS_BELOW_POVERTY \
     FROM CENSUS_DATA \
     ORDER BY PERCENT_HOUSEHOLDS_BELOW_POVERTY DESC \
     LIMIT 5;


 * sqlite:///FinalDB.db
Done.


COMMUNITY_AREA_NAME,PERCENT_HOUSEHOLDS_BELOW_POVERTY
Riverdale,56.5
Fuller Park,51.2
Englewood,46.6
North Lawndale,43.1
East Garfield Park,42.4


### Problem 8
##### Which community area is most crime prone? Display the coumminty area number only.

In [18]:
%sql SELECT COMMUNITY_AREA_NUMBER \
     FROM CHICAGO_CRIME_DATA \
     GROUP BY COMMUNITY_AREA_NUMBER \
     ORDER BY COUNT(*) DESC \
     LIMIT 1;


 * sqlite:///FinalDB.db
Done.


COMMUNITY_AREA_NUMBER
25.0


### Problem 9
##### Use a sub-query to find the name of the community area with highest hardship index

In [19]:
%sql SELECT COMMUNITY_AREA_NAME \
     FROM CENSUS_DATA  \
     WHERE HARDSHIP_INDEX = (  \
         SELECT MAX(HARDSHIP_INDEX) \
     FROM CENSUS_DATA);


 * sqlite:///FinalDB.db
Done.


COMMUNITY_AREA_NAME
Riverdale


### Problem 10
##### Use a sub-query to determine the Community

In [20]:
%sql SELECT COMMUNITY_AREA_NAME  \
     FROM CENSUS_DATA \
     WHERE COMMUNITY_AREA_NUMBER = (  \
         SELECT COMMUNITY_AREA_NUMBER  \
         FROM CHICAGO_CRIME_DATA \
         GROUP BY COMMUNITY_AREA_NUMBER \
         ORDER BY COUNT(*) DESC \
         LIMIT 1 \
     );


 * sqlite:///FinalDB.db
Done.


COMMUNITY_AREA_NAME
Austin


In [21]:
# Display first 5 rows from each table in FinalDB.db

# CENSUS_DATA
census_head = %sql SELECT * FROM CENSUS_DATA LIMIT 5;
print("=== CENSUS_DATA (First 5 Rows) ===")
display(census_head.DataFrame())


 * sqlite:///FinalDB.db
Done.
=== CENSUS_DATA (First 5 Rows) ===


,COMMUNITY_AREA_NUMBER,COMMUNITY_AREA_NAME,PERCENT_OF_HOUSING_CROWDED,PERCENT_HOUSEHOLDS_BELOW_POVERTY,PERCENT_AGED_16__UNEMPLOYED,PERCENT_AGED_25__WITHOUT_HIGH_SCHOOL_DIPLOMA,PERCENT_AGED_UNDER_18_OR_OVER_64,PER_CAPITA_INCOME,HARDSHIP_INDEX
0,1.0,Rogers Park,7.7,23.6,8.7,18.2,27.5,23939,39.0
1,2.0,West Ridge,7.8,17.2,8.8,20.8,38.5,23040,46.0
2,3.0,Uptown,3.8,24.0,8.9,11.8,22.2,35787,20.0
3,4.0,Lincoln Square,3.4,10.9,8.2,13.4,25.5,37524,17.0
4,5.0,North Center,0.3,7.5,5.2,4.5,26.2,57123,6.0


In [22]:
# CHICAGO_PUBLIC_SCHOOLS
schools_head = %sql SELECT * FROM CHICAGO_PUBLIC_SCHOOLS LIMIT 5;
print("\n=== CHICAGO_PUBLIC_SCHOOLS (First 5 Rows) ===")
display(schools_head.DataFrame())

 * sqlite:///FinalDB.db
Done.

=== CHICAGO_PUBLIC_SCHOOLS (First 5 Rows) ===


,School_ID,NAME_OF_SCHOOL,"Elementary, Middle, or High School",Street_Address,City,State,ZIP_Code,Phone_Number,Link,Network_Manager,...,Freshman_on_Track_Rate__,X_COORDINATE,Y_COORDINATE,Latitude,Longitude,COMMUNITY_AREA_NUMBER,COMMUNITY_AREA_NAME,Ward,Police_District,Location
0,610038,Abraham Lincoln Elementary School,ES,615 W Kemper Pl,Chicago,IL,60614,(773) 534-5720,http://schoolreports.cps.edu/SchoolProgressRep...,Fullerton Elementary Network,...,NDA,1171699.458,1915829.428,41.924497,-87.644522,7,LINCOLN PARK,43,18,"(41.92449696, -87.64452163)"
1,610281,Adam Clayton Powell Paideia Community Academy ...,ES,7511 S South Shore Dr,Chicago,IL,60649,(773) 535-6650,http://schoolreports.cps.edu/SchoolProgressRep...,Skyway Elementary Network,...,NDA,1196129.985,1856209.466,41.760324,-87.556736,43,SOUTH SHORE,7,4,"(41.76032435, -87.55673627)"
2,610185,Adlai E Stevenson Elementary School,ES,8010 S Kostner Ave,Chicago,IL,60652,(773) 535-2280,http://schoolreports.cps.edu/SchoolProgressRep...,Midway Elementary Network,...,NDA,1148427.165,1851012.215,41.747111,-87.731702,70,ASHBURN,13,8,"(41.74711093, -87.73170248)"
3,609993,Agustin Lara Elementary Academy,ES,4619 S Wolcott Ave,Chicago,IL,60609,(773) 535-4389,http://schoolreports.cps.edu/SchoolProgressRep...,Pershing Elementary Network,...,NDA,1164504.290,1873959.199,41.809757,-87.672145,61,NEW CITY,20,9,"(41.8097569, -87.6721446)"
4,610513,Air Force Academy High School,HS,3630 S Wells St,Chicago,IL,60609,(773) 535-1590,http://schoolreports.cps.edu/SchoolProgressRep...,Southwest Side High School Network,...,91.8,1175177.622,1880745.126,41.828146,-87.632794,34,ARMOUR SQUARE,11,9,"(41.82814609, -87.63279369)"


In [23]:
# CHICAGO_CRIME_DATA
crime_head = %sql SELECT * FROM CHICAGO_CRIME_DATA LIMIT 5;
print("\n=== CHICAGO_CRIME_DATA (First 5 Rows) ===")
display(crime_head.DataFrame())

 * sqlite:///FinalDB.db
Done.

=== CHICAGO_CRIME_DATA (First 5 Rows) ===


,ID,CASE_NUMBER,DATE,BLOCK,IUCR,PRIMARY_TYPE,DESCRIPTION,LOCATION_DESCRIPTION,ARREST,DOMESTIC,...,DISTRICT,WARD,COMMUNITY_AREA_NUMBER,FBICODE,X_COORDINATE,Y_COORDINATE,YEAR,LATITUDE,LONGITUDE,LOCATION
0,3512276,HK587712,2004-08-28,047XX S KEDZIE AVE,890,THEFT,FROM BUILDING,SMALL RETAIL STORE,0,0,...,9,14.0,58.0,6,1155838.0,1873050.0,2004,41.807440,-87.703956,"(41.8074405, -87.703955849)"
1,3406613,HK456306,2004-06-26,009XX N CENTRAL PARK AVE,820,THEFT,$500 AND UNDER,OTHER,0,0,...,11,27.0,23.0,6,1152206.0,1906127.0,2004,41.898280,-87.716406,"(41.898279962, -87.716405505)"
2,8002131,HT233595,2011-04-04,043XX S WABASH AVE,820,THEFT,$500 AND UNDER,NURSING HOME/RETIREMENT HOME,0,0,...,2,3.0,38.0,6,1177436.0,1876313.0,2011,41.815933,-87.624642,"(41.815933131, -87.624642127)"
3,7903289,HT133522,2010-12-30,083XX S KINGSTON AVE,840,THEFT,FINANCIAL ID THEFT: OVER $300,RESIDENCE,0,0,...,4,7.0,46.0,6,1194622.0,1850125.0,2010,41.743665,-87.562463,"(41.743665322, -87.562462756)"
4,10402076,HZ138551,2016-02-02,033XX W 66TH ST,820,THEFT,$500 AND UNDER,ALLEY,0,0,...,8,15.0,66.0,6,1155240.0,1860661.0,2016,41.773455,-87.706480,"(41.773455295, -87.706480471)"
